In [1]:
import pandas as pd
import altair as alt

from IPython.display import Markdown

_ = alt.renderers.set_embed_options(theme="ggplot2", actions=False)


In [2]:
%load_ext sql
%config SqlMagic.displaycon = False
%sql --connection_arguments '{"read_only": true}'

In [3]:
# %%sql filecoin_state_market_deals= <<

# select
#     *
# from filecoin_state_market_deals
# limit 10


In [5]:
%%sql daily_data_on_chain <<
select
    date_trunc('day', sector_start_at) as date,
    sum(piece_size / 1024 ** 5) as total_stored_data
from filecoin_state_market_deals
where sector_start_epoch > 0 and slash_epoch = -1
group by 1
order by 1


In [6]:
df = daily_data_on_chain.DataFrame()
df["date"] = pd.to_datetime(df["date"])


In [11]:
# Altair chart of daily data on chain (verified vs unverified)

alt.Chart(df).mark_area().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("total_stored_data:Q", title="Total stored data (PiB)"),
    color=alt.value("#55595c"),
    tooltip=["date:T", "total_stored_data:Q"],
).properties(
    width="container",
    height=400,
    title="Data Onboarding Rate",
)


alt.Chart(...)

In [13]:
# Now cumulative data on chain

df["cumulative_data"] = df["total_stored_data"].cumsum()

alt.Chart(df).mark_area().encode(
    x=alt.X("date:T", title="Date"),
    y=alt.Y("cumulative_data:Q", title="Cumulative data on chain (PiB)"),
    color=alt.value("#55595c"),
    tooltip=["date:T", "cumulative_data:Q"],
).properties(
    width="container",
    height=400,
    title="Cumulative data on chain",
)


alt.Chart(...)